In [1]:
import torch
import torch.nn as nn
import numpy as np
import sys
#sys.path.insert(0, '../drbayes')


from subspace_inference.models import PretrainedCIFARModel
from subspace_inference.losses import cross_entropy
from subspace_inference.utils import train_epoch
from sklearn.metrics import accuracy_score
from subspace_inference import utils
from subspace_inference.posteriors import SWAG
from torchvision import transforms
from torch.utils.data import DataLoader

from subspace_inference.posteriors.proj_model import SubspaceModel
from subspace_inference.posteriors.ess import EllipticalSliceSampling
from subspace_inference.posteriors.vi_model import VIModel, ELBO
import math

TypeError: an integer is required (got type bytes)

In [2]:
import sklearn

TypeError: an integer is required (got type bytes)

In [3]:
!pip3 install pytorchcv

In [2]:
DATA_PATH = './data/'

In [3]:
x_train = np.loadtxt(DATA_PATH + "cifar10_train_x.csv")
y_train = np.loadtxt(DATA_PATH + "cifar10_train_y.csv")
x_test = np.loadtxt(DATA_PATH + "cifar10_test_x.csv")
y_test = np.loadtxt(DATA_PATH + "cifar10_test_y.csv")

In [4]:
x_train = x_train.reshape((len(x_train), 32, 32, 3))
x_test = x_test.reshape((len(x_test), 32, 32, 3))

In [5]:
trnsfms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.4914, 0.4822, 0.4465), std=(0.2023, 0.1994, 0.2010)),
])

In [6]:
class MyDataset(torch.utils.data.Dataset):
    def __init__(self, X, y, transforms=None):
        self.X = X
        self.y = y
        self.transforms = transforms
        
    def __len__(self, ):
        return len(self.X)
        
    def __getitem__(self, idx):
        X = self.X[idx].astype(np.float32)
        y = self.y[idx].astype(int)
        if self.transforms is not None:
            X = self.transforms(X)
            
        return X, torch.tensor(y)

In [7]:
trainset = MyDataset(x_train, y_train, transforms=trnsfms)
testset = MyDataset(x_test, y_test, transforms=trnsfms)

In [8]:
batch_size = 100
test_batch_size = 100
num_epochs = 15

In [9]:
train_loader = DataLoader(trainset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(testset, batch_size=test_batch_size, shuffle=False)

In [10]:
class CE_logits:
    def __call__(self, model, input, target):

        output = model(input)
        loss = torch.nn.functional.cross_entropy(output, target)
        
        return loss, output, {}

In [11]:
def train(model, loader, optimizer, criterion, epochs=3000, swag_model=None, swag=False, swag_start=2000,
          print_freq=100):
    
    for epoch in range(epochs):
        train_res = train_epoch(loader, model, criterion, optimizer, cuda=True, regression=False);
        
        if swag and epoch > swag_start:
            swag_model.collect_model(model)
        
        if (epoch % print_freq == 0 or epoch == epochs - 1):
            print('Epoch %d. Loss: %.4f' % (epoch, train_res['loss']))

In [12]:
model_cfg = PretrainedCIFARModel
model_cfg.kwargs = {'name': "resnet20_cifar10", 'pretrained': True, 'spec_norm': True}

In [13]:
swag_model = SWAG(model_cfg.base, subspace_type="pca", *model_cfg.args, **model_cfg.kwargs, 
              subspace_kwargs={"max_rank": 10, "pca_rank": 10});
model = model_cfg.base(*model_cfg.args, **model_cfg.kwargs).cuda()
optimizer = torch.optim.Adam(model.parameters());

In [14]:
criterion = CE_logits()

In [15]:
train(model=model, loader=train_loader, optimizer=optimizer, criterion=criterion, epochs=15, print_freq=1, 
      swag=True, swag_model=swag_model, swag_start=5)

Epoch 0. Loss: 0.3236
Epoch 1. Loss: 0.1772
Epoch 2. Loss: 0.1344
Epoch 3. Loss: 0.1108
Epoch 4. Loss: 0.0872
Epoch 5. Loss: 0.0815
Epoch 6. Loss: 0.0703
Epoch 7. Loss: 0.0610
Epoch 8. Loss: 0.0569
Epoch 9. Loss: 0.0515
Epoch 10. Loss: 0.0512
Epoch 11. Loss: 0.0438
Epoch 12. Loss: 0.0454
Epoch 13. Loss: 0.0430
Epoch 14. Loss: 0.0378


In [16]:
utils.save_checkpoint(
     dir="ckpts",
     epoch=15,
     name="my_swag_checkpoint"+str(15),
     state_dict=swag_model.state_dict(),
)

In [17]:
def get_pca_space():
    swag_model = SWAG(model_cfg.base, subspace_type="pca", *model_cfg.args, **model_cfg.kwargs, 
                  subspace_kwargs={"max_rank": 10, "pca_rank": 10})
    swag_model.load_state_dict(torch.load("ckpts/my_swag_checkpoint15-15.pt")["state_dict"])
    mean, _, cov_factor = swag_model.get_space()
    subspace = SubspaceModel(mean, cov_factor)
    return mean, subspace

In [18]:
mean, subspace = get_pca_space()

In [19]:
from subspace_inference.posteriors.mcmc import ProjSGLD


sgld_model = ProjSGLD(model = model_cfg.base, loader=train_loader, criterion = criterion, mean = mean, subspace = subspace, epochs = 100, cyclic=True, T= 10, lr = 1e-2, **model_cfg.kwargs)

In [32]:
trajectories = []
for batch in test_loader:
    X, y = batch
    samples = sgld_model.sample(1, 10)[0]
    out = ProjectedModel(model = sgld_model.model, subspace= sgld_model.subspace,  proj_params = samples ).forward(X)
    trajectories.append(out.detach().numpy())


    

trajectories = np.vstack(trajectories)

In [33]:
trajectories.shape

(10000, 10)

In [47]:
y_pred = torch.softmax(torch.from_numpy(trajectories), dim=-1).numpy()

In [48]:
with open('neurips_bdl_starter_kit/data/cifar10/probs.csv', 'r') as fp:
    reference = np.loadtxt(fp)

In [49]:
sys.path.append("neurips_bdl_starter_kit")
import metrics

In [50]:
metrics.agreement(y_pred, reference)

0.9152

In [51]:
metrics.total_variation_distance(y_pred, reference)

0.14333060356444047